# **Introduction**

This notebook is for testing stable baselines.

# **Import Packages**

Import the necessary packages for this implementation.


In [4]:
# import these:
import gymnasium as gym
import numpy as np
from stable_baselines3 import SAC

USE:

In [ ]:
env = gym.make("InvertedPendulum-v5")
model = SAC("MlpPolicy", env, verbose = 0)
model.learn(total_timesteps = 50000)

TEST:

In [9]:
# render settings:
width = 1280
height = 1280
default_camera_config = {"azimuth" : 90.0, "elevation" : 0.0, "distance" : 3.5, "lookat" : [0.0, 0.0, 0.25]}

env = gym.make("InvertedPendulum-v5", render_mode = "human", width = width, height = height, default_camera_config = default_camera_config)
obs, _ = env.reset()
done = False

while not done:
    action, _ = model.predict(obs, deterministic = True)
    obs, reward, term, trunc, _ = env.step(action)
    done = term or trunc
    env.render()

c:\Users\mtidd2\Desktop\rl_experimentation\venv\Lib\site-packages\glfw\__init__.py:917: GLFWError: (65537) b'The GLFW library is not initialized'
  warnings.warn(message, GLFWError)


Pressed ESC
Quitting.
